In [1]:
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from selenium import webdriver 

In [2]:
#카테고리 페이지 실행
url = "https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001&gateCd=Drawer&t_page=%EB%93%9C%EB%A1%9C%EC%9A%B0_%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&t_click=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%ED%83%AD_%EB%8C%80%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC&t_1st_category_type=%EB%8C%80_%EC%8A%A4%ED%82%A8%EC%BC%80%EC%96%B4"
driver = webdriver.Chrome("../driver/chromedriver")
driver.get(url)

In [3]:
# 데이터 프레임 생성 
df_review_cos2 = pd.DataFrame(columns=['category', 'ranking' 'name', 'brand', 'price', 'sale_price', 'ranking', 'date', 'rate', 'id', 'skin_type', 'select_1_title', 'select_1_content', 'select_2_title', 'select_2_content', 'select_3_title', 'select_3_content', 'txt'])
df_review_cos2

,category,rankingname,brand,price,sale_price,ranking,date,rate,id,skin_type,select_1_title,select_1_content,select_2_title,select_2_content,select_3_title,select_3_content,txt


In [4]:
# 상품 리뷰 크롤링 함수
def product_crawling(df, page_num):    
        for page in range(1, page_num):      
                # 리뷰의 인덱스는 한 페이지 내에서 1~10까지 존재
                current_page = page
                for i in range(1, 10):  # 한 페이지 내 10개 리뷰 크롤링
                    try:
                        date = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.date').text
                        rate = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.score_area > span.review_point > span').text
                        id = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.info > div > p.info_user > a.id').text
                        skin_type = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
                        select_1_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dt > span').text
                        select_1_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(1) > dd > span').text
                        select_2_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dt > span').text
                        select_2_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(2) > dd > span').text
                        select_3_title = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dt > span').text
                        select_3_content = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.poll_sample > dl:nth-child(3) > dd > span').text
                        txt = driver.find_element(By.CSS_SELECTOR, f'#gdasList > li:nth-child({i}) > div.review_cont > div.txt_inner').text
                        df.loc[len(df)] = [category, ranking, name, brand, price, sale_price, date, rate, id, skin_type, select_1_title, select_1_content, select_2_title, select_2_content, select_3_title, select_3_content, txt]
                    except:
                         pass
                try:
                    if current_page % 10 != 0 and current_page // 10 < 1: # 현재 페이지가 10의 배수가 아니고 10페이지 이하일 때
                        page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({page+1})')
                        page_button.click() # 리뷰 10개 긁으면 next 버튼 클릭
                        time.sleep(3)
                    elif current_page % 10 != 0 and current_page // 10 >= 1: # 현재 페이지가 10의 배수가 아니고 10페이지 이상일 때
                        page_button = driver.find_element(By.CSS_SELECTOR, f'#gdasContentsArea > div > div.pageing > a:nth-child({page+2})')
                        page_button.click() 
                        time.sleep(3)
                    elif current_page % 10 == 0:
                        next_button = driver.find_element(By.CSS_SELECTOR, '#gdasContentsArea > div > div.pageing > a.next')
                        next_button.click() # 현재 페이지가 10의 배수일 때 페이지 넘김 버튼 클릭
                        time.sleep(3)
                except:
                     pass
                print(f'{page}페이지 크롤링 완료')

In [5]:
#전체 크롤링 진행
#품목창 들어가기
review_text = driver.find_element(By.CSS_SELECTOR, f'#Contents > div > div.ct-menu > ul > li:nth-child(2) > a > span').text
review = driver.find_element(By.CSS_SELECTOR, f'#Contents > div > div.ct-menu > ul > li:nth-child(2) > a > span').click()
print(f"현재는 {review_text} 입니다.")

#화장품 품목별 개수 확인
cos_list = driver.find_element(By.CSS_SELECTOR, "#Contents > ul.cate_list_box").text
cos_lists = list(cos_list.split('\n'))

for skincare1 in range(2, len(cos_lists)):
    #항목으로 들어갑니다.
    select_skin = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul.cate_list_box > li:nth-child({skincare1}) > a').click()
    time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
    
    #현재 화장품 종류 확인_이 변수를 데이터 셋에 넣어주면 됩니다.
    category = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul.cate_list_box > li:nth-child({skincare1}) > a').text
    print(f"현재 항목은 {category}입니다.")

    #48개씩 보기
    select_show = driver.find_element(By.CSS_SELECTOR, '#Contents > div.cate_align_box > div.count_sort.tx_num > ul > li:nth-child(3) > a').click()
    time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
    print('항목을 48개씩 보겠습니다.')

    #페이지 리스트 만들기
    next_pages = []
    pre_pages = []
    page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing').text
    pages = list(page.split())
    if '다음' in pages: #다음 페이지 버튼이 있으면 확인합니다.
            select_page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing > a.next').click()
            time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
            
            #다음 페이지의 리스트를 만듭니다.
            next_page = driver.find_element(By.CSS_SELECTOR, '#Container > div.pageing').text
            next_pages = list(next_page.split())
            next_pages = next_pages[3:]
            driver.back() 
    pre_pages = pages[:10]
    pages = pre_pages + next_pages #다음장의 페이지 항목을 더해줍니다.(최대 20페이지가 넘지 않는다는 상황에서 작동합니다.)
    print(f'페이지 항목은 {pages}입니다.')
    # 1페이지 크롤링
    for now_page in range(1, 2): #페이지 항목 수 만큼 반복하여 진행합니다.
        now_pages = int(now_page)
        if now_pages==1 or 10 or 11:  #1,11번 페이지는 다른 주소명을 갖고 있어 pass로 넘겼습니다. 
            pass

        elif now_pages > 10:
            select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
            time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

        else:   #다음 페이지로 넘어가는 코드입니다.
            select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
            time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

        print(f'현재는 {now_pages}페이지 입니다.')
        # 11행 1열부터 12행 4열까지 크롤링합니다.
        ranking = 1     #순위를 매기기 위한 변수입니다.
        for row in tqdm_notebook(range(7, 19)): #총 12개의 행입니다.
            for col in range(1, 5): #총 4개의 열입니다.
                if col == 1: #첫번째 상품은 다른 형식이라 따로 지정
                    product = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({row}) > li.flag > div > a > img').click()
                    reviews = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').click()
                    time.sleep(0.2) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

                    #상품이 클릭된 페이지 입니다. 여기서 상품 정보를 긁어오는 코드를 넣으시면 됩니다.
                    review_cnt = int(driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a > span').text.replace(",", "").replace("(", "").replace(")", ""))
                    name = driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name").text
                    brand = driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_brand").text
                    # 할인이 적용된 제품과 할인이 적용되지 않은 제품의 가격이 다르게 표시되어 있습니다.
                    try: 
                        price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").text.replace(",", ""))
                    except: 
                        price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong").text.replace(",", ""))
                    try:
                        sale_price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong").text.replace(",", ""))
                    except:
                        sale_price = price
                    # 제품 당 최대 리뷰 개수는 1000개 입니다. 1000개가 넘어가면 1000개만 크롤링합니다.
                    if review_cnt != 0:
                        if review_cnt < 1000:
                            product_crawling(df_review_cos2, round(review_cnt/10)+1)
                        else:        
                            product_crawling(df_review_cos2, 101)
                    else:
                        pass
                    print(f"{row-6}행에 {col}열의 항목이 스크랩되었습니다.")
                    ranking += 1
                    try:
                        driver.back()
                    except:
                        pass
                else:
                    product = driver.find_element(By.CSS_SELECTOR, f'#Contents > ul:nth-child({row}) > li:nth-child({col}) > div > a > img').click()
                    reviews = driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a').click()
                    time.sleep(5) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.
                    
                    #상품이 클릭된 페이지 입니다. 여기서 상품 정보를 긁어오는 코드를 넣으시면 됩니다.
                    review_cnt = int(driver.find_element(By.CSS_SELECTOR, '#reviewInfo > a > span').text.replace(",", "").replace("(", "").replace(")", ""))
                    name = driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name").text
                    brand = driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_brand").text
                    try: 
                        price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").text.replace(",", ""))
                    except: 
                        price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong").text.replace(",", ""))
                    try:
                        sale_price = int(driver.find_element(By.CSS_SELECTOR, "#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-2 > strong").text.replace(",", ""))
                    except:
                        sale_price = price

                    if review_cnt != 0:
                        if review_cnt < 1000:
                            product_crawling(df_review_cos2, round(review_cnt/10)+1)
                        else:        
                            product_crawling(df_review_cos2, 101)
                    else:
                        pass
    
                    print(f"{row-6}행에 {col}열의 항목이 스크랩되었습니다.")
                    try:
                        driver.back()
                    except:
                        pass
        
        if now_pages == 10: #10페이지가 끝나면 다음 페이지로 넘어갑니다.
            select_page = driver.find_element(By.CSS_SELECTOR, f'#Container > div.pageing > a:nth-child({now_pages})').click()
            time.sleep(1) #페이지가 넘어가면서 로딩이 느리면 중간에 멈춥니다. 타이머를 조절해주세요.

#초기 페이지로 돌아가기 위해서 시도합니다.
start_page = driver.find_element(By.CSS_SELECTOR, '#btnGnbOpen').click()
start_page = driver.find_element(By.CSS_SELECTOR, '#gnbAllMenu > ul > li:nth-child(1) > div:nth-child(2) > p:nth-child(1) > a').click()
print('작업을 완료하고 초기 페이지로 돌아왔습니다.')

현재는 미스트/오일 입니다.
현재 항목은 미스트/픽서입니다.
항목을 48개씩 보겠습니다.
페이지 항목은 ['1', '2']입니다.
현재는 1페이지 입니다.


  0%|          | 0/12 [00:00<?, ?it/s]

1페이지 크롤링 완료
2페이지 크롤링 완료
3페이지 크롤링 완료
4페이지 크롤링 완료
5페이지 크롤링 완료
6페이지 크롤링 완료
7페이지 크롤링 완료
8페이지 크롤링 완료
9페이지 크롤링 완료
10페이지 크롤링 완료
11페이지 크롤링 완료
12페이지 크롤링 완료
13페이지 크롤링 완료
14페이지 크롤링 완료
15페이지 크롤링 완료
16페이지 크롤링 완료
17페이지 크롤링 완료
18페이지 크롤링 완료
19페이지 크롤링 완료
20페이지 크롤링 완료
21페이지 크롤링 완료
22페이지 크롤링 완료
23페이지 크롤링 완료
24페이지 크롤링 완료
25페이지 크롤링 완료
26페이지 크롤링 완료
27페이지 크롤링 완료
28페이지 크롤링 완료
29페이지 크롤링 완료
30페이지 크롤링 완료
31페이지 크롤링 완료
32페이지 크롤링 완료
33페이지 크롤링 완료
34페이지 크롤링 완료
35페이지 크롤링 완료
36페이지 크롤링 완료
37페이지 크롤링 완료
38페이지 크롤링 완료
39페이지 크롤링 완료
40페이지 크롤링 완료
41페이지 크롤링 완료
42페이지 크롤링 완료
43페이지 크롤링 완료
44페이지 크롤링 완료
45페이지 크롤링 완료
46페이지 크롤링 완료
47페이지 크롤링 완료
48페이지 크롤링 완료
49페이지 크롤링 완료
50페이지 크롤링 완료
51페이지 크롤링 완료
52페이지 크롤링 완료
53페이지 크롤링 완료
54페이지 크롤링 완료
55페이지 크롤링 완료
56페이지 크롤링 완료
57페이지 크롤링 완료
58페이지 크롤링 완료
59페이지 크롤링 완료
60페이지 크롤링 완료
61페이지 크롤링 완료
62페이지 크롤링 완료
63페이지 크롤링 완료
64페이지 크롤링 완료
65페이지 크롤링 완료
66페이지 크롤링 완료
67페이지 크롤링 완료
68페이지 크롤링 완료
69페이지 크롤링 완료
70페이지 크롤링 완료
71페이지 크롤링 완료
72페이지 크롤링 완료
73페이지 크롤링 완료
74페이지 크롤링 완료
75페이지 크롤링 완료
76페이지 크롤링 완료
77페이지 크롤링 완료
78페이지 크롤

In [ ]:
df_review_cos2.info()

In [ ]:
# csv 파일로 저장
df_review_cos2.to_csv('df_review_cos2.csv', index=False, encoding='utf-8-sig')